In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Spark study - 231209") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
2023-12-09 16:24:34,733 WARN util.Utils: Your hostname, KJH resolves to a loopback address: 127.0.1.1; using 172.24.249.89 instead (on interface eth0)
2023-12-09 16:24:34,736 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-12-09 16:24:35,664 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_genome_scores = spark.read.csv("file:///home/kjh/data/ml-latest/genome-scores.csv", header=True, inferSchema=True)
df_genome_tags = spark.read.csv("file:///home/kjh/data/ml-latest/genome-tags.csv", header=True, inferSchema=True)
df_movies = spark.read.csv("file:///home/kjh/data/ml-latest/movies.csv", header=True, inferSchema=True)
df_links = spark.read.csv("file:///home/kjh/data/ml-latest/links.csv", header=True, inferSchema=True)
df_ratings = spark.read.csv("file:///home/kjh/data/ml-latest/ratings.csv", header=True, inferSchema=True)
df_tags = spark.read.csv("file:///home/kjh/data/ml-latest/tags.csv", header=True, inferSchema=True)

In [4]:
df_ratings.show(4)
df_tags.show(4)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
|     1|    110|   4.0|1225865086|
|     1|    158|   4.0|1225733503|
|     1|    260|   4.5|1225735204|
+------+-------+------+----------+
only showing top 4 rows

+------+-------+-------------+----------+
|userId|movieId|          tag| timestamp|
+------+-------+-------------+----------+
|    10|    260| good vs evil|1430666558|
|    10|    260|Harrison Ford|1430666505|
|    10|    260|       sci-fi|1430666538|
|    14|   1221|    Al Pacino|1311600756|
+------+-------+-------------+----------+
only showing top 4 rows



In [5]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps

/home/kjh/.local/share/virtualenvs/kjh-4pV5HF-C/lib/python3.10/site-packages/pyspark/pandas/__init__.py:47: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [6]:
psdf_ratings = df_ratings.to_pandas_on_spark()
psdf_tags = df_tags.to_pandas_on_spark()

In [7]:
psdf_ratings.head()

2023-12-09 16:25:05,042 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:25:05,078 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [8]:
psdf_ratings.describe()

,userId,movieId,rating,timestamp
count,3.383216e+07,3.383216e+07,3.383216e+07,3.383216e+07
mean,1.654380e+05,2.831348e+04,3.542540e+00,1.269362e+09
std,9.534122e+04,4.992865e+04,1.063959e+00,2.541023e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,8.294600e+04,1.219000e+03,3.000000e+00,1.046694e+09
50%,1.661160e+05,3.263000e+03,4.000000e+00,1.264698e+09
75%,2.474610e+05,4.049100e+04,4.000000e+00,1.496886e+09
max,3.309750e+05,2.889830e+05,5.000000e+00,1.689843e+09


In [9]:
psdf_ratings.dtypes

userId         int32
movieId        int32
rating       float64
timestamp      int32
dtype: object

In [10]:
psdf_ratings['rating_datetime'] = ps.to_datetime(psdf_ratings['timestamp'], errors='coerce', unit='s')
psdf_tags['tag_datetime'] = ps.to_datetime(psdf_tags['timestamp'], errors='coerce', unit='s')

In [11]:
psdf_ratings.head()

2023-12-09 16:25:40,688 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:25:40,689 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/home/kjh/.local/share/virtualenvs/kjh-4pV5HF-C/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,userId,movieId,rating,timestamp,rating_datetime
0,1,1,4.0,1225734739,2008-11-03 17:52:19
1,1,110,4.0,1225865086,2008-11-05 06:04:46
2,1,158,4.0,1225733503,2008-11-03 17:31:43
3,1,260,4.5,1225735204,2008-11-03 18:00:04
4,1,356,5.0,1225735119,2008-11-03 17:58:39


In [12]:
psdf_tags.head()

2023-12-09 16:25:51,724 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:25:51,726 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/home/kjh/.local/share/virtualenvs/kjh-4pV5HF-C/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,userId,movieId,tag,timestamp,tag_datetime
0,10,260,good vs evil,1430666558,2015-05-03 15:22:38
1,10,260,Harrison Ford,1430666505,2015-05-03 15:21:45
2,10,260,sci-fi,1430666538,2015-05-03 15:22:18
3,14,1221,Al Pacino,1311600756,2011-07-25 13:32:36
4,14,1221,mafia,1311600746,2011-07-25 13:32:26


In [13]:
psdf_ratings = psdf_ratings.drop('timestamp', axis=1)
psdf_tags = psdf_tags.drop('timestamp', axis=1)

In [14]:
psdf_tags.head()

2023-12-09 16:25:53,139 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:25:53,140 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/home/kjh/.local/share/virtualenvs/kjh-4pV5HF-C/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,userId,movieId,tag,tag_datetime
0,10,260,good vs evil,2015-05-03 15:22:38
1,10,260,Harrison Ford,2015-05-03 15:21:45
2,10,260,sci-fi,2015-05-03 15:22:18
3,14,1221,Al Pacino,2011-07-25 13:32:36
4,14,1221,mafia,2011-07-25 13:32:26


In [17]:
psdf_user = ps.merge(psdf_ratings, psdf_tags, on=['userId', 'movieId'], how='inner')

In [18]:
psdf_user.head()

2023-12-09 16:30:52,119 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:30:52,121 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:30:52,121 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:30:52,129 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:31:01,158 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:31:01,161 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

,userId,movieId,rating,rating_datetime,tag,tag_datetime
0,26,296,4.5,2015-04-18 22:59:59,crime,2015-04-18 23:15:19
1,26,296,4.5,2015-04-18 22:59:59,cult film,2015-04-18 23:15:19
2,26,296,4.5,2015-04-18 22:59:59,quentin tarantino,2015-04-18 23:15:19
3,37,1036,5.0,2020-01-04 18:53:43,action,2020-01-04 18:53:48
4,37,1036,5.0,2020-01-04 18:53:43,Alan Rickman,2020-01-04 18:53:53


In [19]:
psdf_user.info

2023-12-09 16:35:46,617 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:35:46,619 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:35:46,620 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:35:46,622 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:35:56,224 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-12-09 16:35:56,225 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

<bound method DataFrame.info of      userId  movieId  rating     rating_datetime                                 tag        tag_datetime
0        26      296     4.5 2015-04-18 22:59:59                               crime 2015-04-18 23:15:19
1        26      296     4.5 2015-04-18 22:59:59                           cult film 2015-04-18 23:15:19
2        26      296     4.5 2015-04-18 22:59:59                   quentin tarantino 2015-04-18 23:15:19
3        37     1036     5.0 2020-01-04 18:53:43                              action 2020-01-04 18:53:48
4        37     1036     5.0 2020-01-04 18:53:43                        Alan Rickman 2020-01-04 18:53:53
5        37     1036     5.0 2020-01-04 18:53:43                             classic 2020-01-04 18:54:36
6        37     1036     5.0 2020-01-04 18:53:43                               tense 2020-01-04 18:54:14
7        37    69844     4.5 2020-01-04 20:03:48                        Alan Rickman 2020-01-04 20:03:52
8        37    69844   

2023-12-09 16:43:58,498 ERROR cluster.StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
2023-12-09 16:43:58,504 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:919)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$a